In [1]:
import pandas as pd
from datetime import datetime, timedelta
from statsmodels.tsa.api import ExponentialSmoothing
import numpy as np
import  warnings
warnings.filterwarnings('ignore')


In [2]:
# 读取数据
sales_data = pd.read_excel(r'C:\Users\work\Desktop\读书笔记\妈妈杯\题目\B题\附件表\附件1-商家历史出货量表.xlsx')
promo_sales_data = pd.read_excel(r'C:\Users\work\Desktop\读书笔记\妈妈杯\题目\B题\附件表\附件6-促销期间商家出货量表.xlsx')
product_info = pd.read_excel(r'C:\Users\work\Desktop\读书笔记\妈妈杯\题目\B题\附件表\附件2-商品信息表.xlsx')
seller_info = pd.read_excel(r'C:\Users\work\Desktop\读书笔记\妈妈杯\题目\B题\附件表\附件3-商家信息表.xlsx')
warehouse_info = pd.read_excel(r'C:\Users\work\Desktop\读书笔记\妈妈杯\题目\B题\附件表\附件4-仓库信息表.xlsx')
result_table2 = pd.read_excel(r'C:\Users\work\Desktop\读书笔记\妈妈杯\code\B\结果表\结果表2-预测结果表.xlsx')
result_table2 = result_table2.rename(columns={"forecast_qty": "qty"})
# 合并历史需求量数据
sales_data['date'] = pd.to_datetime(sales_data['date'])
promo_sales_data['date'] = pd.to_datetime(promo_sales_data['date'])
result_table2['date'] = pd.to_datetime(result_table2['date'])
historical_demand = pd.concat([sales_data, promo_sales_data,result_table2], ignore_index=True)
historical_demand

,seller_no,product_no,warehouse_no,date,qty
0,seller_19,product_448,wh_30,2023-05-09,10.000000
1,seller_19,product_448,wh_30,2023-04-17,14.000000
2,seller_19,product_448,wh_30,2023-01-09,2.000000
3,seller_19,product_448,wh_30,2023-01-20,1.000000
4,seller_19,product_448,wh_30,2023-02-13,22.000000
...,...,...,...,...,...
355836,seller_19,wh_24,product_408,2023-05-20,187.830622
355837,seller_19,wh_24,product_408,2023-05-21,192.138325
355838,seller_19,wh_24,product_408,2023-05-22,193.874493
355839,seller_19,wh_24,product_408,2023-05-23,194.433655


In [3]:
# 计算基准需求量
historical_demand['date'] = historical_demand['date'].dt.date
historical_demand['demand'] = historical_demand.groupby(['date','seller_no', 'product_no', 'warehouse_no'])['qty'].transform('mean')

# 提取特征信息
merged_data = pd.merge(historical_demand, product_info, on='product_no', how='left')
merged_data = pd.merge(merged_data, seller_info, on='seller_no', how='left')
merged_data = pd.merge(merged_data, warehouse_info, on='warehouse_no', how='left')
merged_data

,seller_no,product_no,warehouse_no,date,qty,demand,category1,category2,category3,seller_category,inventory_category,seller_level,warehouse_category,warehouse_region
0,seller_19,product_448,wh_30,2023-05-09,10.000000,10.000000,手机通讯,手机配件,手机配件_12,数码,C,Large,中心仓,华南
1,seller_19,product_448,wh_30,2023-04-17,14.000000,14.000000,手机通讯,手机配件,手机配件_12,数码,C,Large,中心仓,华南
2,seller_19,product_448,wh_30,2023-01-09,2.000000,2.000000,手机通讯,手机配件,手机配件_12,数码,C,Large,中心仓,华南
3,seller_19,product_448,wh_30,2023-01-20,1.000000,1.000000,手机通讯,手机配件,手机配件_12,数码,C,Large,中心仓,华南
4,seller_19,product_448,wh_30,2023-02-13,22.000000,22.000000,手机通讯,手机配件,手机配件_12,数码,C,Large,中心仓,华南
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
365905,seller_19,wh_24,product_408,2023-05-20,187.830622,187.830622,NaN,NaN,NaN,数码,C,Large,NaN,NaN
365906,seller_19,wh_24,product_408,2023-05-21,192.138325,192.138325,NaN,NaN,NaN,数码,C,Large,NaN,NaN
365907,seller_19,wh_24,product_408,2023-05-22,193.874493,193.874493,NaN,NaN,NaN,数码,C,Large,NaN,NaN
365908,seller_19,wh_24,product_408,2023-05-23,194.433655,194.433655,NaN,NaN,NaN,数码,C,Large,NaN,NaN


In [4]:
merged_data.to_excel(r'C:\Users\work\Desktop\读书笔记\妈妈杯\题目\B题\附件表\merged_data.xlsx',index=False)

In [5]:
merged_data = pd.read_excel(r'C:\Users\work\Desktop\读书笔记\妈妈杯\题目\B题\附件表\merged_data.xlsx')
# 构建时间序列模型并预测

    

In [8]:
forecast_data = pd.DataFrame()

i =0
for group, subset in merged_data.groupby(['date','seller_no', 'product_no', 'warehouse_no']):
    #try
    subset = subset.sort_values('date')  # 确保数据按日期顺序排列
    subset = subset.groupby('date')['demand'].mean()  # 将重复的日期分组并求平均值
    temp_df = pd.DataFrame({'demand':subset})
    temp_df = temp_df.reindex(pd.date_range(start='2022-11-01', end='2023-05-30'), fill_value=0)
    model = ExponentialSmoothing(temp_df['demand'], seasonal_periods=7, trend='add', seasonal='add')
    results = model.fit()
    forecast = results.predict(start='2023-06-01', end='2023-06-20')
    group_data = pd.DataFrame({'seller_no': [group[1]]*len(forecast), 'product_no': [group[2]]*len(forecast), 'warehouse_no': [group[3]]*len(forecast),'date': forecast.index, 'demand': forecast.values})
    forecast_data = pd.concat([forecast_data, group_data], ignore_index=True)
    print(i)
    if i==1000:
        break
    else:
        i +=1
forecast_data = forecast_data.rename({'demand':'qty'})

forecast_data['demand'] = np.where(forecast_data['demand'] <0, 0, np.ceil(forecast_data['demand']))

forecast_data.to_excel(r'C:\Users\work\Desktop\读书笔记\妈妈杯\题目\B题\2023年MathorCup大数据挑战赛-赛道B初赛\附件数据及提交结果表格\结果表\结果表3-预测结果表.xlsx', index=False)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

PermissionError: [Errno 13] Permission denied: 'C:\\Users\\work\\Desktop\\读书笔记\\妈妈杯\\题目\\B题\\2023年MathorCup大数据挑战赛-赛道B初赛\\附件数据及提交结果表格\\结果表\\结果表3-预测结果表.xlsx'